In [9]:
# Import dependencies
import pandas as pd
import time

In [10]:
start_load_time = time.time()
clean_reviews_df = pd.read_csv("imdbReviewsClean.csv", sep="\t")
print(f"Load complete, total time {time.time() - start_load_time} seconds")
clean_reviews_df.head()

Load complete, total time 39.48960542678833 seconds


,movie,rating,review_summary,review_date,spoiler_tag,review_detail,year,title
0,The Droving (2020),2,An honest review,2020-05-03,False,Here's the truth. There's not much to this mov...,2020,The Droving
1,All About Eve (1950),10,Amazing,2020-05-03,False,Having seen this film for the first time today...,1950,All About Eve
2,Runaway Train (1985),7,Impressive action scenes!,2020-05-03,False,The movie had some very impressive scenes. Esp...,1985,Runaway Train
3,The Half of It (I) (2020),4,Needed the other half of the movie to cover up...,2020-05-03,False,I see that Netflix has a teenage/kids audience...,2020,The Half of It (I)
4,Closure (I) (2018),9,Fun and intriguing,2020-05-03,False,This is a fun and intriguing mystery. The acti...,2018,Closure (I)


In [23]:
# Expect we only need to fix review_date
clean_reviews_df["review_date"] = pd.to_datetime(clean_reviews_df["review_date"])
print(clean_reviews_df.dtypes)

movie                     object
rating                     int64
review_summary            object
review_date       datetime64[ns]
spoiler_tag                 bool
review_detail             object
year                       int64
title                     object
dtype: object


In [15]:
# Which movies have the most reviews?
movie_review_value_counts = clean_reviews_df["title"].value_counts()
unique_movie_count = len(movie_review_value_counts)
print(f"{unique_movie_count:,} unique movies\n")
most_reviewed_movie = movie_review_value_counts.head(1).index[0]
most_reviewed_movie_count = movie_review_value_counts.head(1)[0]
print(f"Most reviewed movie in working data: {most_reviewed_movie} with {most_reviewed_movie_count:,} reviews.\n")
print(movie_review_value_counts.head(15))

219,807 unique movies

Most reviewed movie in working data: Avengers: Endgame with 8,673 reviews.

Avengers: Endgame                                    8673
Dil Bechara                                          7735
The Shawshank Redemption                             7720
Captain Marvel                                       7060
Wonder Woman 1984                                    6724
The Dark Knight                                      6620
小丑                                                   6451
Star Wars: Episode VIII - The Last Jedi              6354
Mrs. Serial Killer                                   5377
STAR WARS：天行者的崛起                                     5082
Gunjan Saxena: The Kargil Girl                       4912
Laxmii                                               4714
The Lord of the Rings: The Fellowship of the Ring    4627
Star Wars: Episode VII - The Force Awakens           4627
Joker                                                4199
Name: title, dtype: int64


In [16]:
# How concentrated is review activity?
print("Summary Stats: Count of Reviews per Unique Movie")
print(movie_review_value_counts.describe()[1:]) # "count" below refers to total unique movies 

Summary Stats: Count of Reviews per Unique Movie
mean      18.229638
std      103.015005
min        1.000000
25%        1.000000
50%        3.000000
75%        8.000000
max     8673.000000
Name: title, dtype: float64


In [17]:
# What % of movies recieves more than the avg. amount of reviews?
avg_reviews = movie_review_value_counts.describe()['mean']
above_avg_reviews = len(movie_review_value_counts[movie_review_value_counts > avg_reviews])
print(f"Average review count per unique movie: {avg_reviews:.2f}")
print(f"Movies with above avg. review count: {above_avg_reviews:,} / {unique_movie_count:,} = {above_avg_reviews / unique_movie_count * 100:.2f}%")

Average review count per unique movie: 18.23
Movies with above avg. review count: 29,718 / 219,807 = 13.52%
